Загрузим данные для задачи регрессии.

Это те же данные о стоимости домов в Калифорнии, с которыми мы уже работали.

Будем искать гиперпараметры случайного леса для достижения наилучшего результата в этой задаче.

In [ ]:
from sklearn.datasets import fetch_california_housing
import pandas as pd
import numpy as np

data = fetch_california_housing(as_frame=True)

In [ ]:
X = data.data
y = data.target

X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


Разобьём данные на train и test и обучим сначала лес с параметрами по умолчанию.

In [ ]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=1)

Теперь обучим на тренировочных данных случайный лес с параметрами по умолчанию.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model_rf = RandomForestRegressor()

model_rf.fit(Xtrain, ytrain)

RandomForestRegressor()

Сделаем предсказание на тесте и оценим ошибку MSE.

In [ ]:
from sklearn.metrics import mean_squared_error

pred_rf = model_rf.predict(Xtest)

mean_squared_error(ytest, pred_rf)

0.2604636155402258

Из предыдущего практического примера вы уже знаете, что чем больше деревьев у леса, тем лучше качество. Мы проверили, что начиная со 100 деревьев ошибка уменьшается незначительно.

Давайте возьмём с запасом — 150 деревьев у леса.

In [ ]:
from sklearn.metrics import mean_squared_error

model_rf = RandomForestRegressor(n_estimators = 150)

model_rf.fit(Xtrain, ytrain)

pred_rf = model_rf.predict(Xtest)

mean_squared_error(ytest, pred_rf)

0.2585462668296284

Самые важные на практике гиперпараметры:
*   число деревьев (с ним определились);
*   max_depth — максимальная глубина деревьев;
*   max_features — максимальное число признаков, используемое для поиска оптимального предиката на каждом шаге построения дерева.

Подберём max_depth и max_features при помощи GridSearchCV по тренировочным данным.

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'max_depth' : np.arange(2, 25, 4),
          'max_features' : ['sqrt', 'log2', None]}

gs = GridSearchCV(RandomForestRegressor(n_estimators=150), params, cv=3, scoring='neg_mean_squared_error')

gs.fit(Xtrain, ytrain)

GridSearchCV(cv=3, estimator=RandomForestRegressor(n_estimators=150),
             param_grid={'max_depth': array([ 2,  6, 10, 14, 18, 22]),
                         'max_features': ['sqrt', 'log2', None]},
             scoring='neg_mean_squared_error')

Посмотрим на наилучший набор гиперпараметров и значение метрики для него.

In [ ]:
gs.best_estimator_

RandomForestRegressor(max_depth=22, max_features='log2', n_estimators=150)

In [ ]:
gs.best_score_

-0.2509153368700138

Обучим на всех тренировочных данных лес с наилучшими гиперпараметрами и, наконец, оценим качество модели на тестовых данных.

In [ ]:
model = gs.best_estimator_

model.fit(Xtrain, ytrain)

RandomForestRegressor(max_depth=22, max_features='log2', n_estimators=150)

In [ ]:
pred = model.predict(Xtest)

mean_squared_error(ytest, pred)

0.24493601987243083

Путём подбора гиперпараметров удалось уменьшить ошибку, это успех!

Можно также подбирать и другие гиперпараметры у леса. Однако одновременно подбирать несколько гиперпараметров — очень долго, так как GridSearchCV перебирает всевозможные комбинации гиперпараметров и придётся долго ждать.

Как выход — последовательный подбор гиперпараметров. Например, сейчас мы подобрали n_estimators, max_depth и max_features. И уже с найденными значениями этих гиперпараметров можно подбирать другие, менее важные на практике гиперпараметры.